In [5]:
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as cond

from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, TimeoutException, NoSuchElementException

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager

import re



from dotenv import load_dotenv
from pathlib import Path
import os
from collections import deque


import time

In [8]:
'''
Load user and password, double checks path
'''

path = f'{os.getcwd()}/.env'
load_dotenv(dotenv_path = path)

user = os.getenv('CLIENT_USER')
passwrd = os.getenv('CLIENT_PASS')

In [9]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://learn.zybooks.com/zybook/GSUCSC1302Spring2023')

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:01<00:00, 7.05MB/s]


In [10]:
'''
Block used to login faster, doesn't work anymore. Must login manually after bot browser opens up.
'''


# Select and type in user email into text box
user_email = WebDriverWait(driver, 20).until(
    cond.element_to_be_clickable(
        (By.XPATH, '//*[@id="ember8"]')
    )
)
user_email.send_keys(user)

# Select and type in user password into text box
user_pass = WebDriverWait(driver, 20).until(
    cond.element_to_be_clickable(
        (By.XPATH, '//*[@id="ember10"]')
    )
)
user_pass.send_keys(passwrd)

# Click signin button
sign_in = WebDriverWait(driver, 20).until(
    cond.element_to_be_clickable(
        (By.XPATH, '//*[@id="ember6"]/div/div[3]/button')
    )
)
sign_in.click()

TypeError: object of type 'NoneType' has no len()

In [ ]:
'''
boolean function to check if a problem is completed on a specific webpage.
'''
def is_completed(selenium_element):
    try:
        selenium_element.find_element(By.CLASS_NAME, 'zb-chevron.title-bar-chevron.grey.chevron-outline.large')   # check to see if element is completed
        return False
    except (NoSuchElementException) as e:
        return True
    

In [ ]:
'''
Clicks all radio buttons for every multi-choice question

'''

def complete_MCQ(driver):
    # a block refers to a single problem set on the current page
    activity_blocks = driver.find_elements(By.CLASS_NAME, 'interactive-activity-container.multiple-choice-content-resource.participation.large.ember-view')
    
    # clicks every possible radio button. If the correct answer is selected once, it is marked as correct permanently.
    for blocks in activity_blocks:
        choice_list = blocks.find_elements(By.TAG_NAME, 'input')
        for choice in choice_list:
            choice.send_keys(Keys.RETURN)
            

In [ ]:
# used to run examples for completion, doesn't work usually. 
def run_all_examples(driver):
    button_list = driver.find_elements(By.CLASS_NAME, 'run-button-container')
    for containers in button_list:
        btn = containers.find_element(By.TAG_NAME, 'button')
        btn.send_keys(Keys.RETURN)
        

In [ ]:
# supposed to click the Next button to progress the example program, doesn't work most of the time
def complete_step_program(driver):
    activity_blocks = driver.find_elements(By.CLASS_NAME, 'interactive-activity-container.animation-player-content-resource.participation.large.ember-view')

    for blocks in activity_blocks:
        finished = is_completed(blocks)
        if finished:
            continue
        start_button = WebDriverWait(blocks, 5).until(
            cond.element_to_be_clickable(
                (By.CLASS_NAME, 'title')
            )
        )
        start_button.click()

        while not finished:            
            finished = is_completed(blocks)
            play_btn = WebDriverWait(blocks, 60).until(
                cond.element_to_be_clickable(
                    (By.CLASS_NAME, 'play-button.bounce')
                )
            )
            play_btn.click()
            finished = is_completed(blocks)
        
        

In [ ]:
# clicks 'show answer' then enters the correct answer

def complete_short_answers(driver):          
    activity_block = driver.find_elements(By.CLASS_NAME, "interactive-activity-container.short-answer-content-resource.participation.large.ember-view")

    for blocks in activity_block:
        question_list = blocks.find_elements(By.CLASS_NAME, "question-set-question.short-answer-question.ember-view")        
        
        # clicks all "show me answer" buttons on the page
        for questions in question_list:
            btn = questions.find_element(By.CLASS_NAME, 'zb-button.secondary.show-answer-button') # must be clicked twice to reveal answer
            btn.click()
            btn.click()
        
        # grabs the revealed answer and types it in
        for questions in question_list:
            ans = questions.find_element(By.CLASS_NAME, 'forfeit-answer')
            text_box = questions.find_element(By.TAG_NAME, 'textarea')
            text_box.send_keys(ans.text)
        
        # submits final answer for all questions
        for questions in question_list:
            btn = questions.find_element(By.CLASS_NAME, 'zb-button.primary.raised.check-button')
            btn.click()
            


In [ ]:

def check_assignment_list(driver):
    try:
        back_btn = driver.find_element(By.CLASS_NAME, 'zb-button.secondary.left.icon-button-with-title.mr-auto')
        back_btn.click()
    except (NoSuchElementException) as e:
        return True

In [ ]:
'''
Loads assignments present on the web page based on the current state of the home page, and the entered hw number.
Class HW was typically titled "HW{assignment number}"

'''
def load_assignments(driver, homework_number):
    tabs = driver.find_element(By.CLASS_NAME, 'tabs')
    btn = tabs.find_elements(By.TAG_NAME, 'button')
    btn[2].click()
    
    found = False
    check_assignment_list(driver=driver)
    homework_list = driver.find_elements(By.CLASS_NAME, 'assignment-summary.p-2.flex.justify-between')
    for hw in homework_list:
        hw_title = re.search(r'(?<=HomeWork )\d', hw.text, flags=re.I)
        hw_title = hw_title.group()
        if int(hw_title) == homework_number:
            hw.click()
            found = True
            break
    
    if not found:
        raise Exception('Could not find homework')
    
    assignments = driver.find_elements(By.CLASS_NAME, 'flex.items-center.mb-2')
    
    # loads subchapter links into a single list
    links = []
    for subchapters in assignments:
        href_container = subchapters.find_element(By.TAG_NAME, 'a')
        href = href_container.get_attribute('href')
        links.append(href)
    return links

In [ ]:
assignment_links = load_assignments(driver, 2)  # load in assignment pages for a specific homework  

In [ ]:
i = 0   # set i so that I can restart the while loop at the correct position if required

In [ ]:
# While loop to go through assignment links
while i < len(assignment_links):
    try:
        driver.get(assignment_links[i])
        time.sleep(5)
        complete_short_answers(driver)
        complete_MCQ(driver)
        complete_step_program(driver)
    except:
        # except block to continue looping instead of breaking 
        print(i)
    time.sleep(15)
    
    i += 1


35
36
38
40
46
47
48
49
51
52
53
54
